# First test model

### 1) load ODYM

In [91]:
# Load a local copy of the current ODYM branch:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
import pickle
import pylab


# Specify path to dynamic stock model and to datafile, relative
MainPath = os.path.join('..', 'odym', 'modules')
DataPath = os.path.join('..', 'docs', 'files')
sys.path.insert(0, MainPath) # add ODYM module directory to system path

sys.path.insert(0, os.path.join('..', 'odym', 'modules')) # add ODYM module directory to system path, relative
sys.path.insert(0, os.path.join(os.getcwd(),'..', 'odym', 'modules')) # add ODYM module directory to system path, absolute

import ODYM_Classes as msc # import the ODYM class file
import ODYM_Functions as msf # import the ODYM function file
import dynamic_stock_model as dsm # import the dynamic stock model library

### 2) Reading and formatting input data

The stock data for wind power capacity is obtained from the EU 2020 reference scenario (LDS) is exported and stored as .xslx. A copy of the file is provided on the GitHub repo.

In [71]:
WindPowerFilePath  = os.path.join(DataPath, 'Analysis_1_input.xlsx')

MyGoods = ['Onshore', 'Offshore']
MyYears = list(np.arange(2000,2051))

In [72]:
WindPower_df = pd.read_excel(WindPowerFilePath, sheet_name='Gross_Electricity_Generation',index_col=0)
WindPower_df = WindPower_df.T

# reindex to include all years
WindPower_df_full = WindPower_df.reindex(MyYears)
#interpolate missing years linearly and keep whole numbers
WindPower_df_full = WindPower_df_full.interpolate(method='linear').round().astype(int)

print(WindPower_df_full.head())


WindPowerArray = WindPower_df_full.to_numpy().T # Type of Wind Turbine x Year
print(WindPowerArray.shape)


      Onshore  Offshore
2000        0         0
2001    13206       305
2002    26411       609
2003    39617       914
2004    52822      1218
(2, 51)


The average lifetime per Good (onshore, offshore) is obtained from .... and read from the input data Excel file. 

In [73]:
LifetimeFile = openpyxl.load_workbook(WindPowerFilePath, data_only=True)
Lifetime_Datasheet = LifetimeFile['Average_Lifetime']

Lifetimes = []

for m in range(len(MyGoods)):
    Lifetimes.append(Lifetime_Datasheet.cell(m+1,2).value) # Add lifetime values to list
print(Lifetimes) 

[25, 30]


### 3) Define MFA system

with just 1 element for now (Nd)

First, define a classifications of objects flowing:

In [74]:
ModelClassification  = {} # Create dictionary of model classifications



ModelClassification['Time'] = msc.Classification(Name = 'Time', Dimension = 'Time', ID = 1, 
                                                 Items = MyYears)

ModelClassification['Cohort'] = msc.Classification(Name = 'Age-cohort', Dimension = 'Time', ID = 2,
                                                   Items = MyYears)
# Classification for cohort is used to track age-cohorts in the stock.


ModelClassification['Good'] = msc.Classification(Name = 'Goods', Dimension = 'Good Type', ID = 3,
                                                   Items = MyGoods)
# Classification for good is used for the wind turbine type

ModelClassification['Product'] = msc.Classification(Name = 'Product', Dimension = 'Power', ID = 4,
                                                   Items = ['Capacity'])
# Classification for product is used for the wind power capacity

ModelClassification['Element'] = msc.Classification(Name = 'Elements', Dimension = 'Element', ID = 5, 
                                                    Items = ['Nd'])

# Get model time start, end, and duration:
Model_Time_Start = int(min(ModelClassification['Time'].Items))
Model_Time_End = int(max(ModelClassification['Time'].Items))
Model_Duration = Model_Time_End - Model_Time_Start

Than define the index table:

In [75]:
IndexTable = pd.DataFrame({'Aspect'        : ['Time','Age-cohort','Good','Product','Element'], # 'Time' and 'Element' must be present!
                           'Description'   : ['Model aspect "Time"','Model aspect "Age-cohort"' ,'Model aspect "Good that produces electricity"' ,'Model aspect "Product"', 'Model aspect "Element"'],
                           'Dimension'     : ['Time','Time','Good Type','Power','Element'], # 'Time' and 'Element' are also dimensions
                           'Classification': [ModelClassification[Aspect] for Aspect in ['Time','Cohort','Good','Product','Element']],
                           'IndexLetter'   : ['t','c','g','p','e']}) # Unique one letter (upper or lower case) indices to be used later for calculations.

# Default indexing of IndexTable, other indices are produced on the fly
IndexTable.set_index('Aspect', inplace = True) 

IndexTable

,Description,Dimension,Classification,IndexLetter
Aspect,,,,
Time,"Model aspect ""Time""",Time,<ODYM_Classes.Classification object at 0x00000...,t
Age-cohort,"Model aspect ""Age-cohort""",Time,<ODYM_Classes.Classification object at 0x00000...,c
Good,"Model aspect ""Good that produces electricity""",Good Type,<ODYM_Classes.Classification object at 0x00000...,g
Product,"Model aspect ""Product""",Power,<ODYM_Classes.Classification object at 0x00000...,p
Element,"Model aspect ""Element""",Element,<ODYM_Classes.Classification object at 0x00000...,e


Define the MFA system:

In [76]:
Dyn_MFA_System = msc.MFAsystem(Name = 'TestSystem', 
                      Geogr_Scope = 'EU',               #what is my exact georgraphical scope?
                      Unit = 'GW', 
                      ProcessList = [], 
                      FlowDict = {}, 
                      StockDict = {},
                      ParameterDict = {}, 
                      Time_Start = Model_Time_Start, 
                      Time_End = Model_Time_End, 
                      IndexTable = IndexTable, 
                      Elements = IndexTable.loc['Element'].Classification.Items) # Initialize MFA system

### 4) Inserting data into the MFA system

Define the processes

In [77]:
Dyn_MFA_System.ProcessList = [] # Start with empty process list, only process numbers (IDs) and names are needed.
Dyn_MFA_System.ProcessList.append(msc.Process(Name = 'Environment', ID   = 0))                          #Define different name for environment!
Dyn_MFA_System.ProcessList.append(msc.Process(Name = 'WT use phase'  , ID   = 1))

# Print list of processes:
Dyn_MFA_System.ProcessList

Defining the parameter values for the ... parameters

In [78]:
ParameterDict = {}

#
ParameterDict['S_wt']= msc.Parameter(Name = 'Wind Turbine Capacity', ID = 1, P_Res = 1,
                                       MetaData = None, Indices = 'g,t', 
                                       Values = WindPowerArray, Unit = 'GW')

#
ParameterDict['tau']   = msc.Parameter(Name = 'mean good lifetime', ID = 2, P_Res = 1, 
                                       MetaData = None, Indices = 'g', 
                                       Values = Lifetimes, Unit = 'yr')
ParameterDict['sigma'] = msc.Parameter(Name = 'stddev of mean good lifetime', ID = 3, P_Res = 1,
                                       MetaData = None, Indices = 'g', 
                                       Values = [0.2 * i for i in Lifetimes], Unit = 'yr')      	        #CHECK standard deviation

# Assign parameter dictionary to MFA system:
Dyn_MFA_System.ParameterDict = ParameterDict

print(ParameterDict['S_wt'].Values)

[[     0  13206  26411  39617  52822  66028  79587  93146 106704 120263
  133822 156186 178549 200913 223276 245640 264828 284016 303204 322392
  341580 372499 403418 434337 465256 496175 531628 567081 602535 637988
  673441 693491 713541 733591 753641 773691 786852 800012 813173 826333
  839494 860507 881520 902532 923545 944558 953780 963003 972225 981448
  990670]
 [     0    305    609    914   1218   1523   2274   3024   3775   4525
    5276   7685  10093  12502  14910  17319  23312  29305  35297  41290
   47283  60624  73965  87307 100648 113989 131762 149535 167308 185081
  202854 214629 226404 238179 249954 261729 270594 279459 288323 297188
  306053 313805 321556 329308 337059 344811 347802 350794 353785 356777
  359768]]


The flows and stock changes are as follows:

In [79]:
# Define the flows of the system, and initialise their values:
Dyn_MFA_System.FlowDict['F_0_1'] = msc.Flow(Name = 'Annually deployed capacity', P_Start = 0, P_End = 1,
                                            Indices = 't,g', Values=None)
Dyn_MFA_System.FlowDict['F_1_0'] = msc.Flow(Name = 'Eol goods', P_Start = 1, P_End = 0,
                                            Indices = 't,c,g', Values=None)
Dyn_MFA_System.StockDict['S_1']   = msc.Stock(Name = 'Wind power stock', P_Res = 1, Type = 0,
                                              Indices = 't,g', Values=None)
Dyn_MFA_System.StockDict['dS_1']  = msc.Stock(Name = 'Wind power stock change', P_Res = 1, Type = 1,
                                              Indices = 't,g', Values=None)

Dyn_MFA_System.Initialize_FlowValues() # Assign empty arrays to flows according to dimensions.
Dyn_MFA_System.Initialize_StockValues() # Assign empty arrays to flows according to dimensions.

In [ ]:
# Check whether flow value arrays match their indices, etc. See method documentation.
Dyn_MFA_System.Consistency_Check() 


[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


### 5) Programming a solution to the MFA system

In [89]:
# Fit model parameter 'S_wt' to right shape in FlowDict:
Dyn_MFA_System.StockDict['S_1'].Values[:,:] = Dyn_MFA_System.ParameterDict['S_wt'].Values.transpose()

# loop over all goods (WT Types) to determine the stock-driven outflow
for GoodType in range(0,len(MyGoods)):
    # Create helper DSM for computing the dynamic stock model:
    DSM_Stock = dsm.DynamicStockModel(t = np.array(MyYears),
                                       s = Dyn_MFA_System.ParameterDict['S_wt'].Values[GoodType,:], 
                                       lt = {'Type': 'Normal', 'Mean': [Dyn_MFA_System.ParameterDict['tau'].Values[GoodType]],
                                             'StdDev': [Dyn_MFA_System.ParameterDict['sigma'].Values[GoodType]]})
    DS = DSM_Stock.compute_stock_change()
    # compute the stock per cohort, outflow per cohort and the inflow
    [S_C, O_C, I] = DSM_Stock.compute_stock_driven_model()

    Dyn_MFA_System.StockDict['dS_1'].Values[:,GoodType] = DS
    Dyn_MFA_System.FlowDict['F_1_0'].Values[:,:,GoodType] = O_C
    Dyn_MFA_System.FlowDict['F_0_1'].Values[:,GoodType] = I

### 6) Mass balance check

In [92]:
Bal = Dyn_MFA_System.MassBalance()
print(Bal.shape) # dimensions of balance are: time step x process x chemical element
print(np.abs(Bal).sum(axis = 0)) # reports the sum of all absolute balancing errors by process.

ValueError: einstein sum subscripts string included output subscript 'e' which never appeared in an input